# Profiling TensorFlow Multi GPU Multi Node Training Job with Amazon SageMaker Debugger

이 노트북은 SageMaker 디버거 프로파일링(debugger profiling) 기능이 활성화된 상태에서 TensorFlow 훈련 작업을 생성하는 과정을 안내합니다. Horovod를 사용하여 다중 GPU 다중 노드 훈련을 생성합니다.

### (Optional) Install SageMaker and SMDebug


2020년 12월에 출시된 새로운 디버거 프로파일링 기능을 사용하려면, 최신 버전의 SageMaker 및 SMDebug SDK가 설치되어 있는지 확인하세요. 다음 코드 셀을 사용하여 라이브러리를 업데이트하고 Jupyter 커널을 다시 시작하여 업데이트를 적용합니다.

In [ ]:
import sys
import IPython
install_needed = False  # should only be True once
if install_needed:
    print("installing deps and restarting kernel")
    !{sys.executable} -m pip install -U boto3 sagemaker smdebug
    IPython.Application.instance().kernel.do_shutdown(True)

## 1. Create a Training Job with Debugger Enabled<a class="anchor" id="option-1"></a>

Boto3 SageMaker 클라이언트의 `create_training_job()` 함수를 사용하여 훈련 작업을 시작합니다.

### Start a SageMaker session and retrieve the current region and the default Amazon S3 bucket URI

In [ ]:
import sagemaker

session = sagemaker.Session()
region = session.boto_region_name
bucket = session.default_bucket()
print(region, bucket)

### Upload a training script to the S3 bucket

In [ ]:
import boto3, tarfile

source = "source.tar.gz"
project = "debugger-boto3-profiling-test"

tar = tarfile.open(source, "w:gz")
tar.add("entry_point/tf-hvd-train.py")
tar.close()

s3 = boto3.client("s3")
s3.upload_file(source, bucket, project + "/" + source)

In [ ]:
upload_file_path = f"s3://{bucket}/{project}/{source}"
print(upload_file_path)

### Create a Boto3 SageMaker client object

In [ ]:
sm = boto3.Session(region_name=region).client("sagemaker")

### Configure the request body of the `create_training_job()` function

`create_training_job()` 함수에 대한 request body에 포함하려면 다음 파라메터들이 필요합니다.

- `TrainingJobName` - 수정하려는 경우 접두사(prefix) 또는 전체 이름을 지정합니다.
- `HyperParameters` - 다음 항목을 설정합니다.
  - `sagemaker_program` 및 `sagemaker_submit_directory` - 훈련 스크립트의 S3 버킷 URI입니다. 이를 통해 SageMaker는 URI에서 훈련 스크립트를 읽고 훈련 작업을 시작할 수 있습니다.
  - `sagemaker_mpi` - 분산 학습을 설정하려면 이러한 키-값(key-value) 쌍을 구성합니다.
  - 모델에 다른 하이퍼파라메터를 추가할 수도 있습니다.
- `AlgorithmSpecification` - `TrainingImage`를 지정합니다. 이 예에서는 공식 TensorFlow DLC 이미지가 사용됩니다. 여기에서 여러분의 고유한 훈련 컨테이너 이미지를 사용할 수도 있습니다.
- `RoleArn` - **다음 셀을 실행하려면 훈련에 사용할 올바른 SageMaker 실행 역할 ARN을 지정해야 합니다.**.
- `DebugHookConfig` 및 `DebugRuleConfigurations` - loss 값과 loss가 감소하지 않는 문제를 감시하도록 사전 설정되어 있습니다.
- `ProfilerConfig` 및 `ProfilerRuleConfigurations` -  시스템 및 프레임워크 메트릭을 수집하고 모든 프로파일링 규칙을 시작하며 디버거 프로파일링 리포트를 생성하도록 사전 설정되어 있습니다.

**중요**: `DebugRuleConfigurations` 및 `ProfilerRuleConfigurations`의 경우 **다음 셀을 실행하려면, [Amazon SageMaker Debugger Registry URLs for Built-in Rule Evaluators](https://docs.aws.amazon.com/sagemaker/latest/dg/debugger-docker-images-rules.html)에서 올바른 디버거 규칙 이미지 URI를 지정해야 합니다.** `sagemaker.debugger.get_rule_container_image_uri(region)` 함수는 디버거 규칙 도커 이미지를 자동으로 검색합니다.

예:
- `us-east-1` 리전일 경우, 올바른 image URI는 **503895931360**.dkr.ecr.**us-east-1**.amazonaws.com/sagemaker-debugger-rules:latest 입니다.
- `us-west-2` 리전일 경우, 올바른 image URI는 **895741380848**.dkr.ecr.**us-west-2**.amazonaws.com/sagemaker-debugger-rules:latest 입니다.


In [ ]:
import datetime

training_job_name = "profiler-boto3-" + datetime.datetime.now().strftime("%Y-%m-%d-%H-%M-%S")

sm.create_training_job(
    TrainingJobName=training_job_name,
    HyperParameters={
        "sagemaker_program": "entry_point/tf-hvd-train.py",
        "sagemaker_submit_directory": "s3://" + bucket + "/" + project + "/" + source,
        "sagemaker_mpi_custom_mpi_options": "-verbose -x HOROVOD_TIMELINE=./hvd_timeline.json -x NCCL_DEBUG=INFO -x OMPI_MCA_btl_vader_single_copy_mechanism=none",
        "sagemaker_mpi_enabled": "true",
        "sagemaker_mpi_num_of_processes_per_host": "4",
    },
    AlgorithmSpecification={
        "TrainingImage": "763104351884.dkr.ecr."
        + region
        + ".amazonaws.com/tensorflow-training:2.4.1-gpu-py37-cu110-ubuntu18.04",
        "TrainingInputMode": "File",
        "EnableSageMakerMetricsTimeSeries": False,
    },
    # You must specify your SageMaker execution role ARN here
    RoleArn=sagemaker.get_execution_role(),
    OutputDataConfig={"S3OutputPath": "s3://" + bucket + "/" + project + "/output"},
    ResourceConfig={"InstanceType": "ml.p3.8xlarge", "InstanceCount": 2, "VolumeSizeInGB": 30},
    StoppingCondition={"MaxRuntimeInSeconds": 86400},
    DebugHookConfig={
        "S3OutputPath": "s3://" + bucket + "/" + project + "/debug-output",
        "CollectionConfigurations": [
            {"CollectionName": "losses", "CollectionParameters": {"train.save_interval": "50"}}
        ],
    },
    DebugRuleConfigurations=[
        {
            "RuleConfigurationName": "LossNotDecreasing",
            # You must specify the correct image URI from https://docs.aws.amazon.com/sagemaker/latest/dg/debugger-docker-images-rules.html
            "RuleEvaluatorImage": sagemaker.debugger.get_rule_container_image_uri(region),
            "RuleParameters": {"rule_to_invoke": "LossNotDecreasing"},
        }
    ],
    ProfilerConfig={
        "S3OutputPath": "s3://" + bucket + "/" + project + "/profiler-output",
        "ProfilingIntervalInMilliseconds": 500,
        "ProfilingParameters": {
            "DataloaderProfilingConfig": '{"StartStep": 5, "NumSteps": 3, "MetricsRegex": ".*", }',
            "DetailedProfilingConfig": '{"StartStep": 5, "NumSteps": 3, }',
            "PythonProfilingConfig": '{"StartStep": 5, "NumSteps": 3, "ProfilerName": "cprofile", "cProfileTimer": "total_time"}',
            "LocalPath": "/opt/ml/output/profiler/",  # Optional. Local path for profiling outputs
        },
    },
    ProfilerRuleConfigurations=[
        {
            "RuleConfigurationName": "ProfilerReport",
            # You must specify the correct image URI from https://docs.aws.amazon.com/sagemaker/latest/dg/debugger-docker-images-rules.html
            "RuleEvaluatorImage": sagemaker.debugger.get_rule_container_image_uri(region),
            "RuleParameters": {"rule_to_invoke": "ProfilerReport"},
        }
    ],
)

## 2.  Analyze Profiling Data


### Install the SMDebug client library to use Debugger analysis tools

In [ ]:
import pip


def import_or_install(package):
    try:
        __import__(package)
    except ImportError:
        pip.main(["install", package])


import_or_install("smdebug")

### Use SMDebug to retrieve saved output data and use analysis tools

훈련이 아직 진행 중인 동안 SageMaker Studio 또는 노트북에서 성능 데이터를 시각화할 수 있습니다. 디버거는 타임라인 차트 또는 히트맵 형식으로 시스템 메트릭을 그리는 유틸리티를 제공합니다. 자세한 내용은 노트북 [profiling_interactive_analysis.ipynb](analysis_tools/profiling_interactive_analysis.ipynb)를 확인하세요. 다음 코드 셀에서 총 CPU 및 GPU 사용률을 시계열 차트로 표시합니다. I/O, 메모리, 네트워크와 같은 다른 메트릭을 시각화하려면, `select_dimension` 및 `select_events`에 전달된 목록을 확장하기만 하면 됩니다.

In [ ]:
from smdebug.profiler.analysis.notebook_utils.training_job import TrainingJob

tj = TrainingJob(training_job_name, region)
tj.wait_for_sys_profiling_data_to_be_available()

In [ ]:
from smdebug.profiler.analysis.notebook_utils.timeline_charts import TimelineCharts

system_metrics_reader = tj.get_systems_metrics_reader()
system_metrics_reader.refresh_event_file_list()

view_timeline_charts = TimelineCharts(
    system_metrics_reader,
    framework_metrics_reader=None,
    select_dimensions=["CPU", "GPU"],
    select_events=["total"],
)

## 3. Download Debugger Profiling Report

프로파일링 리포트 규칙은 기본 규칙 요약 및 다음 단계 권장 사항이 포함된 html 리포트 `profiler-report.html`을 생성합니다. 이 리포트는 S3 버킷에서 찾을 수 있습니다.

In [ ]:
rule_output_path = (
    "s3://"
    + bucket
    + "/"
    + project
    + "/output/"
    + training_job_name
    + "/rule-output/ProfilerReport/profiler-output/"
)

In [ ]:
! aws s3 ls {rule_output_path} --recursive

In [ ]:
! aws s3 cp {rule_output_path} . --recursive

In [ ]:
from IPython.display import FileLink

display("Click link below to view the profiler report", FileLink("profiler-report.html"))

**Note: JupyterLab을 사용하는 경우 리포트를 연 후 왼쪽 상단에서 `Trust HTML` 를 클릭해야 합니다.**